In [9]:
import pandas as pd
import numpy as np

In [10]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
def clean(data, is_training=True):
    # Dropping irrelevant columns
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1) 
    
    # Filling missing values
    cols = ["Pclass", "SibSp", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data.Embarked.fillna(data['Embarked'].mode()[0], inplace=True)
    
    if is_training:
        # Handle outliers in 'Fare' for training data only
        Q1 = data['Fare'].quantile(0.25)
        Q3 = data['Fare'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data = data[(data['Fare'] >= lower_bound) & (data['Fare'] <= upper_bound)]
    
    # Normalize numerical columns
    data['Fare'] = (data['Fare'] - data['Fare'].min()) / (data['Fare'].max() - data['Fare'].min())
    data['Age'] = (data['Age'] - data['Age'].min()) / (data['Age'].max() - data['Age'].min())
    return data

data = clean(data, is_training=True)
test = clean(test, is_training=False)


In [12]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,0.271174,1,0,0.111538,S
2,1,3,female,0.321438,0,0,0.121923,S
3,1,1,female,0.434531,1,0,0.816923,S
4,0,3,male,0.434531,0,0,0.123846,S
5,0,3,male,0.346569,0,0,0.130128,Q


In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)
    
data.head(5)

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,0.271174,1,0,0.111538,2
2,1,3,0,0.321438,0,0,0.121923,2
3,1,1,0,0.434531,1,0,0.816923,2
4,0,3,1,0.434531,0,0,0.123846,2
5,0,3,1,0.346569,0,0,0.130128,1


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Y = data["Survived"]
X = data.drop("Survived", axis=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, Y_train)

In [17]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(Y_val, predictions)

0.7419354838709677

In [18]:

submission_preds = clf.predict(test)


In [19]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                   "Survived":submission_preds
                  })

In [20]:
df.to_csv("submission.csv", index=False)